In [ ]:
##The point of this script is to calculate indivisual and average permeabiliteis for
#for 4-5 repeats

In [ ]:
import MDAnalysis as mda 
import numpy as np
import MDAnalysis.core.groups
import MDAnalysis.analysis.distances
import matplotlib.pyplot as plt 
import MDAnalysis.analysis.rdf as rdf_calc 
from MDAnalysis.analysis.dihedrals import Ramachandran
import numpy.linalg as LA
import lipyphilic
import time
from lipyphilic.leaflets.assign_leaflets import AssignLeaflets
import pandas as pd
#import nglview as nv
import matplotlib.cm as cm
import pandas as pd
import scipy 
from tqdm import tqdm

In [ ]:
#u = MDAnalysis.Universe('/media/favour/LaCie/collab_big/repeats/out/cmld4/prod.gro', '/media/favour/LaCie/collab_big/repeats/out/cmld4/prod.xtc')
u = mda.Universe('/media/favour/LaCie/small_molecules/repeats/out/dkp4/plot1d/prod.tpr', '/media/favour/LaCie/small_molecules/repeats/out/dkp4/plot1d/prod.xtc')

In [ ]:
area_of_membrane = []
volume_of_box = []
i = 0
times= []
for ts in u.trajectory:
    times.append(ts.time/1000)
    area_of_membrane.append(u.dimensions[0]/10 * u.dimensions[1]/10)
    volume_of_box.append(u.dimensions[0]/10 * u.dimensions[1]/10 * u.dimensions[2]/10)
    if i % 10000 == 0:
        print(f'frame {i} at '+str(time.strftime("%H:%M:%S", time.localtime())))
    i += 1 

In [ ]:
print(len(area_of_membrane))
area_average = (np.mean(area_of_membrane))*(10**(-14)) # this is in cm^2
volume_average = (np.mean(volume_of_box)) * ((10**(-21))) #nm^3 to cm^3
print('the average area of the membrane is:', area_average, 'cm^2')
print('the volume average is:',volume_average,'cm^3')

In [ ]:
resname = 'DKP'
mol = u.select_atoms(f'resname {resname}')
first_residue_of_mol = mol.resids[0]
residue_length = len(u.select_atoms(f'resname {resname} and resid {first_residue_of_mol}'))


In [ ]:
#num_events = 20 #5-over 500ns #If we look at total events up and down, need a factor of 1/2 in the permeability function
t = np.max(times)
avogadro = scipy.constants.Avogadro #A guess?
area = area_average
solute_concentration = (len(u.select_atoms('resname DKP').resids)/residue_length) / (volume_average)#*avogadro)
print('the solute concentration is',solute_concentration,'cm^-3' )

In [ ]:
import numpy as np

# CONSTANT
t = 2e-6  # time in seconds (you can change this if needed)
#N is the number of crossings, which is the average crossings from Atomix
# Inputs for 5 systems: N (crossings), A (cm^2), ΔC (mol/cm^3)
systems = [
    {'N':32 , 'A':3.4841144353246584e-13  , 'delta_C':2.9736921672453358e+19 }, 
    {'N': 27, 'A':3.485457910536148e-13 , 'delta_C':2.9737330068657746e+19 },
    {'N': 26,  'A':3.487154781565217e-13   ,'delta_C':2.979311772267589e+19 },
    {'N': 37, 'A':3.478686811670658e-13 , 'delta_C': 2.9750416628941378e+19 },
    #{'N': 293, 'A':3.32738491678248e-13 , 'delta_C':3.32363778511186e+19},
]

# Lists to hold permeability and uncertainties
permeabilities = []
uncertainties = []

# Calculate P and σ_P for each system
for sys in systems:
    N = sys['N']
    A = sys['A']
    delta_C = sys['delta_C']
    
    P = N / (2 * A * t * delta_C)  # permeability in cm/s
    sigma_P = np.sqrt(N) / (2 * A * t * delta_C)
    
    permeabilities.append(P)
    uncertainties.append(sigma_P)

# Convert to numpy arrays for easier math
permeabilities = np.array(permeabilities)
uncertainties = np.array(uncertainties)

# Calculate combined (unweighted) mean permeability
P_mean = np.mean(permeabilities)

# Calculate combined uncertainty (standard error)
n = len(permeabilities)
sigma_combined = np.sqrt(np.sum(uncertainties**2)) / n

# Output
print("Permeability results for each system (cm/s):")
for i, (P, sigma) in enumerate(zip(permeabilities, uncertainties), 1):
    print(f"  System {i}: P = {P:.2e} ± {sigma:.2e}")

print(f"\nCombined Permeability: {P_mean:.2e} cm/s")
print(f"Combined Uncertainty:  ±{sigma_combined:.2e} cm/s")